## **Special Methods** - _Dunder Methods_

In Python we have some special methods and attributes named Dunders (Double Underscore), they are also called Magic methods. They all have specific roles they play. They generally look like `__method()__` this convention is used so as to differentiate them from our own methods avoiding collision or overriding methods.

|Special Methods| Description|
|---------------|------------|
|`__init__()`   | Constructor or initializer in Python classes      |
|`__str__()` and `__repr__()`      | Provide string representation for objects the first is user friendly and the second is developer friendly       |
|`__call__()`| Makes the insance of a class callable |
|`__len__()`| supports the `len()` function|

This is just some of the popular ones we will go ahead to describe more. In general special methods help you to write better OOP code.

#### **`__init__()`**

This works as an object constructor or initializer, it allows class user to provide initial values to any instance attribute defined in the class which is crucial to defining the object.

In [1]:
class Point:
    def __init__(self, x, y):        
        self.x = x
        self.y = y

point = Point(21, 42)
print(point.x)
print(point.y)


21
42


when you create a `point` instance python automatically calls the `__init__()` method under the hood, using the same arguements you passed into the class constructor. So we don't need to call the `__init__()` method ourselves.

#### **`__new__()`**

When we call the class constructor to create a new object/ instance python automatically calls the `__new__()` method as a first step in the instantiation process. This method creates and returns a blank or empty new object of our underlying class. Then Python passes the just created object to `__init__()` for initialization (setting instances attribute and underlying logic). In most cases we won't need to write our own custom `.__new__()` method (or in otherword override the `__new__()` in built in python) as python handles that internally unless we are customizing immutable types like `tuple` or `string` classes, Controlling how instance are creating (implementing singletons or caching...etc) or we wish to skip/ bypass the `__init__()` for some reason, say we want the instance attribute to be initialized in a different way from our our `__init__()` those it... Giving an advantage of havinf something like multiple ways to initialize the object but with only one `__init__()` method.

Let say in our `point` class we also want to be able to create a point object from polar coordinatea and not just rectangular coordinates.

In [ ]:
import math
class point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    @classmethod
    def from_polar(cls, radius, angle):
        _instance = cls.__new__(cls)
        _instance.x = radius * math.cos(angle)
        _instance.y = radius * math.sin(radius)
        return _instance

        

So basically what we are doing here is call `__new__` to create a blank/ empty instance of point by bypassing `__init__`. Then we manually assign  attributes (x and y) to the empty instance, then retun the customized instnace. we did this because `__init__` required rectangular coordinates arguments (x and y), but we also want to be able to use radius and angle. Hence enabling us to construct the object in a completely custom way.




So how do we use the `.__new__()` method to create **immutable types** ?

We know if we are creating immutable types like `tuples, strings or int`,once these object are created we want to make sure they can't be modified. That means we can't use the `__init__()` to set attributes (think this way... "in order to make sure these object can not be re-set with the init method"). So to customize immutable objects we have to do this in `__new__()` instead. This is because `__new__()` will create new object instead of initialize (which can be leavaraged to mutate object).

let's make an immutable point object!

In [13]:
class Point(tuple):
    def __new__(cls, x, y):
        # Create the tuple with the values (x, y)
        return super().__new__(cls, (x, y))

    @property
    def x(self):
        return self[0]

    @property
    def y(self):
        return self[1]

    


In [14]:
p = Point(3, 4)

print(p)          # Point(x=3, y=4)
print(p[0])       # 3
print(p.x)        # 3
print(p.y)        # 4



(3, 4)
3
3
4


In [15]:
# Try to mutate it:
p.x = 10          # ❌ AttributeError
p[0] = 10         # ❌ TypeError: 'Point' object does not support item assignment


AttributeError: property 'x' of 'Point' object has no setter

we used `__new__()` to pass (x, y)  as the tuple content and return a tuple object which is immutable.


How do we use `__new__` to create **Singletons** ??

**What is a Singleton ?**

A singleton is a class that allows only one instance to be created. Any attempt to instantiate it again shouldd return the same object that was initially created.
This is useful when we want:
1. A **Single point of access** (like a configuration manager, logger, or database connection).

2. To control resources and avoid uneccesarry duplication. 

`__new__()` helps us to control how and when a new object/ instnace is created. 

In [21]:
class Singleton:
    _instance =  None
    _initialized = False

    def __new__(cls, *args, **kwargs):
        if cls._instance is None:
            print("creating a new instance ....")
            cls._instance = super().__new__(cls)
        else:
            print("Using existing instance ...")

        return cls._instance
    def __init__(self, value):
        if not self._initialized:
            print("Initializing instance")
            self.value = value
            self._initialized = False
    
        

So what's happening here ?
The first time we call `Singleton(...)` `__new__()` sees `_instance` is None and proceeds to create the new instance usinf `super.__new__()`, then stores it in `_instance`,  after this we can use `__init__()` to set the object attribute provided that it has not be set before (consequently we are saying object has not been instantiated before). Any subsequent calls just return `_instance` (no new object is created because `_initialized` is set to False `__init__()` will not be called.)

In [22]:
a = Singleton("first")
print(a.value)  # first

b = Singleton("second")
print(b.value)  # second (WARNING: __init__ ran again!)

print(a is b)   


creating a new instance ....
Initializing instance
first
Using existing instance ...
Initializing instance
second
True


we can use this to configure all loggings in a multi-module application to go to a single file

```python
import logging

class LoggerSingleton:
    _instance = None

    def __new__(cls, *args, **kwargs):
        if not cls._instance:
            cls._instance = super().__new__(cls)
            cls._instance._initialize_logger()
        return cls._instance

    def _initialize_logger(self):
        self.logger = logging.getLogger("AppLogger")
        self.logger.setLevel(logging.DEBUG)

        if not self.logger.hasHandlers():  # Avoid adding duplicate handlers
            handler = logging.StreamHandler()
            formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
            handler.setFormatter(formatter)
            self.logger.addHandler(handler)

    def get_logger(self):
        return self.logger

```

you might not want to use the Singleton pattern to build a logger:

1. if you are writing a simple script or in a small module
2. Using a framework (like Flask or Django) that already manages logging.
3. if multi-process (not multi-thread) logging, as processes might want to write logs at the same time (this can lead to file or data curroption or incomplete logs and **Race Conditions or Concurrency Issues**)-  then you need process **safe** handing (or `QueueHandler`/ `pipe` which allows multiple processes to queue log messges, a `log listener` process can read from the queue and write to a log file ensuring that logs are written in the correct order withtout confliction) 

#### **`__repr___()`** - _Developer-Friendly Respresentations_

using the `__repr__()` allows us to return the object as a nicely printed string. This is targeted at a devoloper using your code to make object look nice and readable.

In [25]:
import math
class point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    @classmethod
    def from_polar(cls, radius, angle):
        _instance = cls.__new__(cls)
        _instance.x = radius * math.cos(angle)
        _instance.y = radius * math.sin(radius)
        return _instance

    
    def __repr__(self):
        return f"points(x= {self.x}, y= {self.y})"

In [26]:
p = point(2,3)
print(p)

points(x= 2, y= 3)


In [28]:
p = point.from_polar(4, 30)
print(p)

points(x= 0.6170057995503362, y= -3.027209981231713)


### **Operator Overloading in classes with special methods**

Internally Python supports opertions with special method like `(+)` with `.__add__()`

**Operation Overlaoding** involves customizing operations like (+, -, *, /, //, **, ==, ..etc) for certain use case or conditions.

For example:


In [29]:
class Vector:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __add__(self, other):
        return Vector(self.x + other.x, self.y + other.y)

    def __repr__(self):
        return f"Vector({self.x}, {self.y})"

v1 = Vector(2, 3)
v2 = Vector(4, 1)
print(v1 + v2)  # Vector(6, 4)


Vector(6, 4)


**Right side Operation**

Every operator has a rightside version (e.g `__add__, __mul__, __sub__` have `__radd__`, `__rmul__` and `__rsub__` respectively)

the rightside operator works this way: 

for `__radd__` -  _“If I’m on the right side of a + and the left side doesn’t know how to add me, this is how to add myself to the left operand.”_

to get a grasp of things consider this piece of code:



In [30]:
class Number:
    def __init__(self, value):
        self.value = value
    
    def __add__(self, other):
        # Only handle addition with another Number instance
        if isinstance(other, Number):
            return Number(self.value + other.value)
        return NotImplemented  # Signal Python to try __radd__

    def __radd__(self, other):
        # Handle addition when Number is on the right and other is int
        if isinstance(other, int):
            return Number(self.value + other)
        return NotImplemented

    def __repr__(self):
        return f"Number({self.value})"

n = Number(5)

print(n + Number(3))  # Calls n.__add__(Number(3)) => Number(8)
print(10 + n)         # int.__add__(Number(5)) returns NotImplemented, so calls n.__radd__(10) => Number(15)


Number(8)
Number(15)


But we don't actually need this If statements has python know how to handle rightside operators.

In [31]:
class Number:
    def __init__(self, value):
        self.value = value

    def __add__(self, other):
        return Number(self.value + other.value)

    def __radd__(self, other):
        # other might be int or something else
        return Number(self.value + other)

    def __repr__(self):
        return f"Number({self.value})"

n = Number(5)
print(n + Number(3))  # Number(8)  -> calls __add__
print(10 + n)         # Number(15) -> calls __radd__


Number(8)
Number(15)


Recall python functions are first class functions so we can actually use functions as variables. so we can do something like this:

In [32]:
class MyNumber:
    def __init__(self, value):
        self.value = value

    def __repr__(self):
        return f"MyNumber({self.value})"

# This is the function we will assign to __radd__ dynamically
def dynamic_radd(self, other):
    # Assume other is int here for simplicity
    print(f"dynamic_radd called: {other} + {self.value}")
    return MyNumber(other + self.value)

# Assign the function to __radd__ dynamically (after class is defined)
MyNumber.__radd__ = dynamic_radd

n = MyNumber(10)

# Now try adding an int + MyNumber
result = 5 + n  # This calls n.__radd__(5)
print("Result:", result)


dynamic_radd called: 5 + 10
Result: MyNumber(15)


when ever you want to perform operation overloading make reference to this [article](https://realpython.com/python-magic-methods/) 

### **Contolling access to Attributes**

we can use dunder methods to retrieve, set and delete attributes. 

When we set an attribute to a value we usually use the assignment operator (=). When Python detects the assignment, it calls the `.__setattr__()` magic method automatically.

With this method we can customize certain aspects of the assignment process. Say we want to make sure that when instantiating a Circle object we want to make sure the radius specified is positive. We can do something like this:


In [34]:
import math

class Circle:
    def __init__(self, radius):
        self.radius = radius
    
    def __setattr__(self, name, value):
        if name == "radius":
            if not isinstance(value, int|float):
                raise TypeError("radius must be a number")
        if value <= 0:
            raise ValueError("radius must be positive")
        super().__setattr__(name, value)

### **Descriptors**

We can use Descriptors to manage attributes. Python descriptors allows us to add function like behaviours on top of attributes for a given class. They allow us to manage attribute acess abd modification. we use them when we need more control over how attributes are retrieved, set or deleted.

|Method |Description |
|------------------|----------------|
|`__get__(self, instance, type= None)`| Getter method that can allow you to retriecve the current value of an attribute|
|`__set__(self, instance, vlaue)`|Setter method that allows you to set a new value to the managed attribute|
|`__set_name__(self, owner, name)`| Name setter method that allows you to define a name for the managed attribute|
|`__delete__(self, instance)`| Deleter method that allows you to remove thr managed attribute from an object|

remember our product class from OOP:

In [35]:
class Product:
    def __init__(self, base_price):
        self._base_price =  base_price
        self._discount = 0

    @property
    def price(self):
        return self._base_price * (1 - self._discount/ 100)
    
    @property
    def discount(self):
        return self._discount
    @discount.setter
    def discount(self, value):
        self._discount =  value

another way to create getter and setters is by using descriptor classes:

In [36]:
class DiscountDescriptor:
    def __get__(self, instance, owner):
        return instance._discount
    
    def __set__(self, instance, value):
        if value < 0 or value > 100:
            raise ValueError("Discount must be between 0 and 100.")
        instance._discount = value


class PriceDescriptor:
    def __get__(self, instance, owner):
        return instance._base_price * (1 - instance._discount/100)
    
    def __set__(self, instance, value):
        raise AttributeError("Price cannot be directly set. It is calculated based on base price and discount.")
        instance._discount = value

class Product:
    def __init__(self, base_price):
        self._base_price = base_price
        self._discount = 0

    # Applying the descriptors to manage attributes
    price = PriceDescriptor()  # price is calculated based on base_price and discount
    discount = DiscountDescriptor()  # discount is manually set with validation



In [ ]:
product = Product(100)  # base price is 100
print(f"Initial price: {product.price}")  # calls the get method and Should calculate price based on the default discount (0%)

product.discount = 20  # Setting discount to 20%
print(f"Price after discount: {product.price}")  # Should show price after 20% discount

try:
    product.discount = -5  # This should raise an error because discount can't be negative
except ValueError as e:
    print(e)


Initial price: 100.0
Price after discount: 80.0
Discount must be between 0 and 100.


So overall we see that using the `@property` decorator is a convinient way to use getter and setter for an attribute, but it doesn't provide as much flexibilty as descriptors

### **Making your Object callable**

### **Implementing Custom maps and containers**